**Process of obtaining & ranking papers:
1)Use scopus search API to obtain relevant papers, take the scopus id from here.
2)Use abstract retrieval API to obtain relevant metadata by using the obtained scopus ids from step 1.
3)Use the metadata to rank the papers based on the ranking algorithm.
4)Store these papers in a dictionary.

To-do:
1) Ask group how many articles to return? (Module 1)
2) Key is hardcoded
2) Ranking algorithm

1)Scopus Search API - Used to search for abstracts from Scopus database
Resource: http://api.elsevier.com/documentation/SCOPUSSearchAPI.wadl
*Create account on Elsevier to get API key

*Specifying what data to filter: http://api.elsevier.com/documentation/search/SCOPUSSearchViews.htm

*Examples: http://kitchingroup.cheme.cmu.edu/blog/2015/04/03/Getting-data-from-the-Scopus-API/

In [70]:
import requests
import json
from pprint import pprint

API_KEY = "1b1115545b64df3c0cefc15bb7baff8b"
#API key temp. hardcoded, can change later.
#Change 'count' in 'resp' in order to change num of articles returned.
resp = requests.get("http://api.elsevier.com/content/search/scopus?query=SRCTITLE(networks) \
                    &field=dc:identifier,prism:isbn,dc:title,citedby-count,prism:coverDate,dc:creator, \
                    prism:publicationName, dc:description&count=5",
                    headers={'Accept':'application/json', 'X-ELS-APIKey': API_KEY})

results = resp.json()

Scopus_ids = [] #List of scopus_ids, actual data needed

#Certain fields null, need to handle errors properly

#For printing purposes only(to see articles fetched)
for r in results['search-results']['entry']:
    print("================================")
    pprint('title: ' + str(r['dc:title']))
    pprint('publicationDate: ' + str(r['prism:coverDate']))
    pprint('Scopus ID: ' + str(r['dc:identifier'])) #Scopus ID, use with Abstract Retrieval API. (refer below)
    pprint('Citation Count: ' + str(r['citedby-count']))
    pprint('Scopus URL: ' + str(r['prism:url'])) 
    
    try:
        pprint(str(r['prism:isbn']))
    except:
        print("isbn: Null")
    try:
        pprint(str(r['dc:creator']))
    except:
        print("creator: Null")
    
    try:
        pprint(str(r['prism:publicationName']))
    except:
        print("publicationName: Null")
    
    try:
        pprint(str(r['dc:description']))
    except:
        print("Abstract: Null")
    
    Scopus_ids.append(str(r['dc:identifier']))

print('======================================')
print("List of Scopus IDs:\n")
print(Scopus_ids)

'title: Over-time measurement of triadic closure in coauthorship networks'
'publicationDate: 2017-12-01'
'Scopus ID: SCOPUS_ID:85016834353'
'Citation Count: 0'
'Scopus URL: http://api.elsevier.com/content/abstract/scopus_id/85016834353'
isbn: Null
'Kim J.'
publicationName: Null
Abstract: Null
'title: Vulnerability of clustering under node failure in complex networks'
'publicationDate: 2017-12-01'
'Scopus ID: SCOPUS_ID:85014623213'
'Citation Count: 0'
'Scopus URL: http://api.elsevier.com/content/abstract/scopus_id/85014623213'
isbn: Null
'Kuhnle A.'
publicationName: Null
Abstract: Null
('title: Data warehouse design approaches from social media: review and '
 'comparison')
'publicationDate: 2017-12-01'
'Scopus ID: SCOPUS_ID:85011841605'
'Citation Count: 0'
'Scopus URL: http://api.elsevier.com/content/abstract/scopus_id/85011841605'
isbn: Null
'Moalla I.'
publicationName: Null
Abstract: Null
('title: A trust-based collaborative access control model with policy '
 'aggregation for online 

2)Abstract Retrieval API - Used to retrieve a Scopus abstract document using a Scopus ID/Identifier.

Resource: http://api.elsevier.com/documentation/AbstractRetrievalAPI.wadl
Examples:http://kitchingroup.cheme.cmu.edu/blog/2015/04/03/Getting-data-from-the-Scopus-API/

**Metadata list index: (use this to extract metadata for ranking use)
0-authors (multiple authors are seperated by '+'s)
1-title
2-publication name
3-volume
4-page range or article number
5-publication date
6-abstract
7-doi
8-citation count

In [74]:
import requests
import json
from pprint import pprint

API_KEY = "1b1115545b64df3c0cefc15bb7baff8b" #hardcoded

#Used to obtain an article's metadata.
#Must supply article's Scopus_ID as input argument
def get_article_info(SCOPUS_ID):
    
    #Define what data should be obtained from API
    url = "http://api.elsevier.com/content/abstract/scopus_id/" + SCOPUS_ID + "?field=authors,"+\
    "title,publicationName,volume,issueIdentifier,prism:pageRange,coverDate,"+\
    "article-number,doi,citedby-count,prism:aggregationType,description"

    resp = requests.get(url,headers={'Accept':'application/json',
                                     'X-ELS-APIKey': API_KEY})

    results = json.loads(resp.text)
    #'''
    #Formatting the data into list
    fstring = '{authors}spl*{title}spl*{journal}spl*{volume}spl*{articlenum}spl*({date})spl*Abstract: {description}spl*{doi}spl*{cites}'
    metadata = fstring.format(authors=' + '.join([au['ce:indexed-name'] for au in results['abstracts-retrieval-response']['authors']['author']]),
                          title=results['abstracts-retrieval-response']['coredata']['dc:title'],
                          journal=results['abstracts-retrieval-response']['coredata']['prism:publicationName'],
                          volume=results['abstracts-retrieval-response']['coredata']['prism:volume'],
                          articlenum=(results['abstracts-retrieval-response']['coredata'].get('prism:pageRange') or
                              results['abstracts-retrieval-response']['coredata'].get('article-number')),
                          description=results['abstracts-retrieval-response']['coredata']['dc:description'],   
                          date=results['abstracts-retrieval-response']['coredata']['prism:coverDate'],
                          doi='doi:' + results['abstracts-retrieval-response']['coredata']['prism:doi'],
                          cites=int(results['abstracts-retrieval-response']['coredata']['citedby-count'])).split("spl*")
    
    return(metadata)
    #'''

#testing
metadata = get_article_info('SCOPUS_ID:85014623213')
print(metadata) #All metadata
#print(metadata[1]) #Gets abstract


['Kuhnle A. + Nguyen N.P. + Dinh T.N. + Thai M.T.', 'Vulnerability of clustering under node failure in complex networks', 'Social Network Analysis and Mining', '7', '8', '(2017-12-01)', 'Abstract: © 2017, Springer-Verlag Wien.Robustness in response to unexpected events is always desirable for real-world networks. To improve the robustness of any networked system, it is important to analyze vulnerability to external perturbation such as random failures or adversarial attacks occurring to elements of the network. In this paper, we study an emerging problem in assessing the robustness of complex networks: the vulnerability of the clustering of the network to the failure of network elements. Specifically, we identify vertices whose failures will critically damage the network by degrading its clustering, evaluated through the average clustering coefficient. This problem is important because any significant change made to the clustering, resulting from element-wise failures, could degrade ne

3)Ranking algorithm

In [ ]:
#Ranking Algorithm

4)Store in dictionary

In [ ]:
#Store ranked papers in dictionary